In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chess
import chess.pgn
import seaborn as sns
import time
import random
import math
from datetime import datetime
from chessboard import display 
import plotly.express as px


# sys.path.append('/../scripts/')
# from pgn_to_df import pgn_to_df
sns.set()
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10)
plt.style.use('ggplot')

EDA - clean and modify dataset
End product - using flash/flask, create a history match analysis chessboard w/ heatmap
To Do:
Research how to use heatmap for chess
Use other datasets with proper format(PGN)


In [9]:
# Functions go over here

# function "convert_date" and "convert_time" change the format of the former dataset given to be readable
def convert_date(time):
    dt_object = datetime.utcfromtimestamp(time / 1000.0)
    date_component = dt_object.strftime('%Y-%m-%d')
    time_component = dt_object.strftime('%H:%M:%S')
    return date_component


def convert_time(time):
    dt_object = datetime.utcfromtimestamp(time / 1000.0)
    date_component = dt_object.strftime('%Y-%m-%d')
    time_component = dt_object.strftime('%H:%M:%S')
    return time_component

# format the move list for Stockfish to be able to read
def format_moves(moves):
    move_list = moves.split()
    coordinate_moves = []

    for move in move_list:
        # Extract file and rank information from algebraic notation
        file_char = move[0]
        rank_char = move[1]

        # Convert file character to column number (a=1, b=2, ..., h=8)
        file_number = ord(file_char) - ord('a') + 1

        # Append the corresponding coordinate to the result
        coordinate_moves.append(f"{file_char}{rank_char}")
    print(coordinate_moves)
    return " ".join(coordinate_moves)


In [3]:
df = pd.read_csv("games.csv")
df_ = df.copy()
df

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,EfqH7VVH,True,1.499791e+12,1.499791e+12,24,resign,white,10+10,belcolt,1691,jamboger,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...,A80,Dutch Defense,2
20054,WSJDhbPl,True,1.499698e+12,1.499699e+12,82,mate,black,10+0,jamboger,1233,farrukhasomiddinov,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...,A41,Queen's Pawn,2
20055,yrAas0Kj,True,1.499698e+12,1.499698e+12,35,mate,white,10+0,jamboger,1219,schaaksmurf3,1286,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...,D00,Queen's Pawn Game: Mason Attack,3
20056,b0v4tRyF,True,1.499696e+12,1.499697e+12,109,resign,white,10+0,marcodisogno,1360,jamboger,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...,B07,Pirc Defense,4


In [5]:
# initializing important variables used for later
moves = df_['moves']

user_list = pd.concat([df_['white_id'], df_['black_id']]).value_counts()

# Note: ratings may vary
rating_bin = [0, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2300, 2400, 2500, 2700]
labels = ["Novices", "E", "D", "C", "B", "A", "CM", "NM", "FM", "IM", "GM"]


e4 c5 Nf3 Qa5 a3


In [22]:
df_["elo_diff"] = df_['white_rating'] - df_['black_rating']
df_.head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply,elo_diff
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5,309
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4,61
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3,-4
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3,-15
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5,54


In [43]:
# (elo_diff < 0 and winner == 'black')
df_upsets = (
    df_.query("(elo_diff < 0) or (victory_status != 'outoftime')"
             )
    .reset_index(drop=True)
    .copy()
)

#update columns here to add former columns(optional)
upset_columns = ['white_id', 'white_rating', 'black_id', 'black_rating', 'elo_diff', 'winner', 'victory_status', 'turns', 'moves']
upsets = df_upsets[upset_columns]
upsets

,white_id,white_rating,black_id,black_rating,elo_diff,winner,victory_status,turns,moves
0,ischia,1496,a-00,1500,-4,white,mate,61,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...
1,daniamurashov,1439,adivanov2009,1454,-15,white,mate,61,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...
2,daniel_likes_chess,1413,soultego,2108,-695,black,resign,9,e4 Nc6 d4 e5 d5 Nce7 c3 Ng6 b4
3,daniel_likes_chess,1381,subham777,1867,-486,black,resign,38,e4 e6 d4 d5 e5 c5 c3 Nc6 Nf3 Qb6 Be3 Qxb2 Nbd2...
4,daniel_likes_chess,1381,roman12342005,1936,-555,black,resign,60,e4 e6 Nf3 d5 exd5 exd5 Qe2+ Be7 Nc3 Nf6 d4 O-O...
...,...,...,...,...,...,...,...,...,...
8861,jamboger,1252,uwe_liem,1290,-38,white,resign,11,c4 Nf6 Nc3 d5 cxd5 Nxd5 e4 Nf4 d4 c6 Bxf4
8862,jamboger,1222,mateuslichess,1262,-40,white,mate,73,c4 d5 e3 dxc4 Bxc4 Nc6 a3 Ne5 d4 Nxc4 Nc3 e5 N...
8863,jamboger,1219,samael88,1250,-31,white,resign,37,c4 e6 d4 b6 Nc3 Bb7 Nf3 g6 h4 Bg7 Bg5 f6 Bf4 d...
8864,jamboger,1219,schaaksmurf3,1286,-67,white,mate,35,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...


In [44]:
upsets = upsets.sort_values(by="elo_diff")
upsets

,white_id,white_rating,black_id,black_rating,elo_diff,winner,victory_status,turns,moves
2444,burks,1016,lance5500,2621,-1605,black,mate,48,Nf3 Nf6 d4 g6 Bg5 Bg7 Nc3 d5 Qd3 Bf5 Qb5+ Nc6 ...
2443,burks,1016,lance5500,2621,-1605,black,mate,40,Nc3 c5 Nf3 Nc6 Nb5 Nf6 c4 g6 Ng5 Bg7 d3 O-O Bf...
5301,dinobzik,1214,lance5500,2621,-1407,black,resign,58,e4 c5 Nf3 d6 d4 Nf6 Nc3 cxd4 Nxd4 Nc6 Nxc6 bxc...
5024,crazyscientist1,1015,unicornz,2305,-1290,black,resign,22,e4 g6 d4 Bg7 Nc3 d6 f4 Nf6 Nf3 O-O Bd3 c5 e5 d...
5567,raulelbueno,1384,lance5500,2621,-1237,black,resign,22,f3 e5 g3 d5 Bg2 Nf6 b3 Be7 Na3 O-O Bb2 Nc6 Bc3...
...,...,...,...,...,...,...,...,...,...
2071,ayan4you,1234,userx100,1235,-1,black,mate,60,e4 c6 c3 d5 f3 Nf6 Bd3 e5 Qb3 Be6 exd5 Bxd5 c4...
8492,godk2001,1156,iamaliveandbored,1157,-1,draw,draw,7,e4 e5 Qh5 d6 Nf3 Qf6 Bc4
1889,amateur_pawn_star,1467,mk83,1468,-1,white,resign,79,c4 e5 Nc3 Nf6 g3 Bb4 Bg2 Bxc3 bxc3 Nc6 d4 d6 c...
6831,imadogarfarf,1850,die_uhr,1851,-1,white,mate,129,e4 c5 Nf3 Nc6 d4 cxd4 Nxd4 Nxd4 Qxd4 d6 Qd5 e6...


In [45]:
df_upsets = (
    upsets.query("elo_diff < -500"
             )
    .reset_index(drop=True)
    .copy()
)
df_upsets

,white_id,white_rating,black_id,black_rating,elo_diff,winner,victory_status,turns,moves
0,burks,1016,lance5500,2621,-1605,black,mate,48,Nf3 Nf6 d4 g6 Bg5 Bg7 Nc3 d5 Qd3 Bf5 Qb5+ Nc6 ...
1,burks,1016,lance5500,2621,-1605,black,mate,40,Nc3 c5 Nf3 Nc6 Nb5 Nf6 c4 g6 Ng5 Bg7 d3 O-O Bf...
2,dinobzik,1214,lance5500,2621,-1407,black,resign,58,e4 c5 Nf3 d6 d4 Nf6 Nc3 cxd4 Nxd4 Nc6 Nxc6 bxc...
3,crazyscientist1,1015,unicornz,2305,-1290,black,resign,22,e4 g6 d4 Bg7 Nc3 d6 f4 Nf6 Nf3 O-O Bd3 c5 e5 d...
4,raulelbueno,1384,lance5500,2621,-1237,black,resign,22,f3 e5 g3 d5 Bg2 Nf6 b3 Be7 Na3 O-O Bb2 Nc6 Bc3...
...,...,...,...,...,...,...,...,...,...
541,tal_mikhail777,1573,ridics,2076,-503,black,resign,30,d4 Nf6 Nc3 g6 e4 d6 Bf4 Bg7 f3 O-O Bc4 Nc6 a3 ...
542,ozguragarr,1426,raf48,1929,-503,black,resign,54,e4 c6 d4 d5 Bd3 dxe4 Bxe4 Nf6 Bf3 Bf5 Ne2 e6 O...
543,madhaus,997,happycuber,1500,-503,black,resign,45,e4 d5 e5 e6 d4 Nc6 Nf3 Bb4+ c3 Ba5 a3 Nge7 b4 ...
544,ranchowangdu,1500,seanysean,2003,-503,white,resign,70,c4 e5 d3 h6 Nf3 Nc6 a3 a5 Nc3 Nf6 e3 d6 Be2 Be...


In [46]:
df_upsets.to_csv('upset_dataset.csv', index=False)